
## MDP
MDP, or Markov Decision Process : is the main formalism used in Reinforcement Learning


**fully observable MDPs** :  the current state that is given to the agent completely characterizes the environment. So the way in which the environment unfolds depends on some state, and we are told that state.”

**partially observed** can be converted to MDP and so behave largely the same way

Starting with:
Markov Chains >> Markov Reward Processes >>  Markov Decision Processes

### Markov chains


<img src="../image-9.png" width="500px" />


We don’t need to know the full history of states to know what will happen next, just the current one

State transition matrix, probability of transition from a state (row) to another one (col):

<img src="../image-10.png" width="500px" />


A Markov Chain has **no rewards or actions yet**, it simply defines a random process that generates a set of states, each with the Markov property

<img src="../image-11.png" width="500px" />


Student MDP:

<img src="../image-12.png" width="500px" />


One can generate random set of states. Ex: ```['C1', 'C2', 'C3', 'Pub', 'C1', 'C2', 'Sleep']````

Agent make no decision, only at mercy of chance.  

In [1]:
import numpy as np

In [2]:
state_map = {
    "C1": "Class1",
    "C2": "Class2",
    "C3": "Class3",
    "FB": "Facebook",
    "Pub": "Pub",
    "Sleep": "Sleep",
    "Pass": "Pass",
}

states = ["Class1", "Class2", "Class3", "Facebook", "Pub", "Sleep", "Pass"]
terminal_states = ["Sleep", "Pass"]

transitions = {
    "Class1": [
        (0.5, "Class2"),  # Study
        (0.5, "Facebook"),  # Facebook
    ],
    "Class2": [
        (0.8, "Class3"),  # Study
        (0.2, "Sleep"),  # Sleep
    ],
    "Class3": [
        (0.6, "Pass"),
        (0.4, "Pub"),
    ],
    "Facebook": [
        (0.9, "Facebook"),  # Keep browsing
        (0.1, "Class1"),  # Quit
    ],
    "Pub": [
        (0.2, "Class1"),  # Leave pub, go to C1 (0.5 * 0.2)
        (0.4, "Class2"),  # Leave pub, go to C2 (0.5 * 0.4)
        (0.4, "Class3"),  # Leave pub, go to C3 (0.5 * 0.4)
    ],
    "Pass": [],  # Terminal state
    "Sleep": [],  # Terminal state
}


P = {s: [] for s in states}
for s, trans in transitions.items():
    # s = state_map[s_short]
    P[s] = [(p, s_next) for p, s_next in trans]

for t in terminal_states:
    P[t] = [(1.0, t)]  # we stay at terminal states

P

{'Class1': [(0.5, 'Class2'), (0.5, 'Facebook')],
 'Class2': [(0.8, 'Class3'), (0.2, 'Sleep')],
 'Class3': [(0.6, 'Pass'), (0.4, 'Pub')],
 'Facebook': [(0.9, 'Facebook'), (0.1, 'Class1')],
 'Pub': [(0.2, 'Class1'), (0.4, 'Class2'), (0.4, 'Class3')],
 'Sleep': [(1.0, 'Sleep')],
 'Pass': [(1.0, 'Pass')]}

### Markov Reward Processes
A Markov reward process is exactly like a Markov chain, except each step taken generates a reward.

<img src="../image-13.png" width="500px" />
<div/>

<img src="../image-14.png" width="500px" />


trajectories we sample will have an associated return value.

<img src="../image-15.png" width="500px" />


Gamma (0..1) determines how important future rewards are to the agent. if close to 0, consider immediate reward only.

Gamma=1 (for the rest of course for convenience)

Value function: expectation over future rewards 


<img src="../image-16.png" width="500px" />


Thanks to the law of large numbers, a rough estimate of the true value function for a state can be achieved by sampling many trajectories starting from that state, and averaging the sampled returns


In [5]:
r = {
    "Class1": -2.0,
    "Class2": -2.0,
    "Class3": -2.0,
    "Facebook": -1.0,
    "Pub": 1.0,
    "Sleep": 0.0,
    "Pass": 10.0,
}
r

{'Class1': -2.0,
 'Class2': -2.0,
 'Class3': -2.0,
 'Facebook': -1.0,
 'Pub': 1.0,
 'Sleep': 0.0,
 'Pass': 10.0}

In [6]:
def get_transitions(state):
    """Transition function for the MDP"""
    return P[state]


def reward(state) -> float:
    """Reward function for the MDP"""
    return float(r[state])

In [7]:
def step(state):
    """Take a step in the Markov Chain from current state"""
    if state in terminal_states:
        return state

    if state not in transitions:
        raise ValueError(f"Invalid state: {state}")

    # Get possible transitions
    possible_transitions = transitions[state]

    if not possible_transitions:
        return state

    # Sample next state based on probabilities
    probs = [t[0] for t in possible_transitions]
    idx = np.random.choice(len(possible_transitions), p=probs)
    _, next_state = possible_transitions[idx]

    return next_state


step("Class1")  # Example step

'Facebook'

In [10]:
gamma = 1.0
theta = 1e-12
max_iters = 30_000

V = {s: 0.0 for s in states}

# Terminal states yield their reward once and then the episode ends.
# So we set their values directly, and do not update them in the loop.
for t in terminal_states:
    V[t] = reward(t)

for _ in range(max_iters):
    delta = 0.0

    for s in states:
        if s in terminal_states:
            continue
        v_old = V[s]
        exp_next = sum(p * V[s_next] for p, s_next in get_transitions(s))
        V[s] = reward(s) + gamma * exp_next
        delta = max(delta, abs(v_old - V[s]))
    if delta < theta:
        break

V

{'Class1': -12.543209876532266,
 'Class2': 1.4567901234579068,
 'Class3': 4.320987654322317,
 'Facebook': -22.543209876523473,
 'Pub': 0.8024691358056364,
 'Sleep': 0.0,
 'Pass': 10.0}

#### Bellman equation



<img src="../image-17.png" width="500px" />



as gamma is constant:


<img src="../image-18.png" width="500px" />
<div/>

<img src="../image-19.png" width="500px" />
<div/>

<img src="../image-20.png" width="500px" />


So : **value of a state s is the expectation over the immediate reward plus the discounted expected future reward***


##### How do we get that expected reward of the next state v(St+1)?



<img src="../image-21.png" width="500px" />


= sum of all state-values weighted by their probability of occurring. Many of the values in this sum will be zero, because you can only transition to some states at each step

Notice that the immediate reward is no longer expressed as an expectation because it’s a constant, and the expectation of a constant is just a constant, so we can pull it out of the expectation.

**Bellman equation** : expresses the state-value in terms of itself .

<img src="../image-22.png" width="500px" />

Nodes b1 and b2 are possible next states reachable from node a



we’re taking the immediate reward r and then averaging over the values of all possible successor states

Self check with former value calculated iteratively:

<img src="../image-23.png" width="500px" />

Different way to solve it:

##### Analytical solution


<img src="../image-24.png" width="500px" />
<div/>

<img src="../image-25.png" width="500px" />

<div/>
<img src="../image-26.png" width="500px" />

Often to large to be used





In [11]:
## Analytical Value Calculation with bellman Equations


_rewards = [-2, -2, -2, 10, 1, -1, 0]
p_matrix = [
    [0, 0.5, 0, 0, 0, 0.5, 0],
    [0, 0, 0.8, 0, 0, 0, 0.2],
    [0, 0, 0, 0.6, 0.4, 0, 0],
    [0, 0, 0, 0, 0, 0, 1],
    [0.2, 0.4, 0.4, 0, 0, 0, 0],
    [0.1, 0, 0, 0, 0, 0.9, 0],
    [0, 0, 0, 0, 0, 0, 0],
]
gamma = 1
R = np.array(_rewards)
P = np.matrix(p_matrix)
I = np.identity(len(p_matrix))
solution = np.dot(np.linalg.inv((I - gamma * P)), R)
solution = solution.tolist()[0]
for state in range(len(states)):
    print(states[state], solution[state])


Class1 -12.543209876543214
Class2 1.4567901234567908
Class3 4.320987654320986
Facebook 10.0
Pub 0.8024691358024674
Sleep -22.543209876543223
Pass 0.0


##### Iterative solutions

Do it later

### Markov Decision Processes (MDP)

It introduces actions. The inclusion of actions means that the transition probabilities as well as the rewards must now be **conditioned on a chosen action**


<img src="../image-27.png" width="500px" /><div/>


There is now a separate state transition matrix and reward for every action

<img src="../image-28.png" width="500px" /><div/>

For this particular MDP, every action save for “Pub” is deterministic. In the case of the action “ Pub”, the agent no longer enters into the “Pub” state, but instead transitions with some probability into one of the 3 “Class” states.

Why reduce the “Pub” state to an action instead? To show that actions can have both deterministic and stochastic effects on where the agent ends up in the environment.

we also don’t need the “Pass” state anymore. Whereas in the MC and MRP examples we only got rewards for exiting states, we now get rewards after taking actions, so we can get the +10 reward for studying in Class 3 without needing to transition into a “Pass” state that immediately takes us to “Sleep”

#### Policies

Agent policy:  a function that tells the agent what action to take depending on the state (or a probability distribution to sample from). Policy can also be deterministic (no sampling)


<img src="../image-29.png" width="500px" /><div/>

Later: Choosing a policy impacts value function. Calculate value function under many policies and select policy with better value.


##### Any MDP to MRP

We can reformulate any MDP into a MRP (which we know how to solve analytically). Change MDP to be action independent. 

The rewards in the MDP are conditioned on which action is taken, so we need to average over all actions from a single state to get a single action-independent reward value. Each state has an expected reward for any action

<img src="../image-30.png" width="300px" /><div/>
<img src="../image-31.png" width="300px" /><div/>
<img src="../image-32.png" width="500px" /><div/>


In [14]:
# Transform MDP to a MRP

# Probabilities changed to reflect uniform random policy
# Notice Class 3 probabilities reflect possible "Pub" choice:
# (.5 * .2) = probability of picking pub (.5) AND
# probability of then being sent to class 1 (.2)
# Together they mean a total probability of (.5 * .2) = .1
# for ending up back in C1 from C3

state_names = ["C1", "C2", "C3", "FB", "Sleep"]

p_matrix = [
    [0, 0.5, 0, 0.5, 0],
    [0, 0, 0.5, 0, 0.5],
    [0.1, 0.2, 0.2, 0, 0.5],
    [0.5, 0, 0, 0.5, 0],
    [0, 0, 0, 0, 0],
]
# Action rewards are also weighted and summed by probability of occurring
# I.E: 5.5 = (.5 * 10) + (.5 * 1)
_rewards = [-1.5, -1, 5.5, -0.5, 0]
gamma = 1
R = np.array(_rewards)
P = np.matrix(p_matrix)
I = np.identity(len(p_matrix))
solution = np.dot(np.linalg.inv((I - gamma * P)), R)
solution = solution.tolist()[0]
for state in range(len(state_names)):
    print(state_names[state], solution[state])

C1 -1.307692307692308
C2 2.6923076923076925
C3 7.384615384615385
FB -2.3076923076923075
Sleep 0.0



#### MDP Value Functions

<img src="../image-33.png" width="500px" /><div/>

The state-value function tells us how much reward we can expect from the current state onward, if we follow the chosen policy.

<img src="../image-34.png" width="500px" /><div/>

we can use an average over many samples to estimate the action-value function


#### MDP Bellman Equation

<img src="../image-35.png" width="500px" /><div/>
<img src="../image-36.png" width="500px" /><div/>



They represent the same underlying value: the expected future reward

<img src="../image-37.png" width="500px" /><div/>


**States** = Open Circles; **Actions** = Closed Circles

State-value function is the average of all the available action-values, weighted by how likely we are to choose them under our current policy:

<img src="../image-38.png" width="500px" /><div/>

Similarly, the action-value function is the average value of the states the agent might transition to given the chosen action, but weighted instead by their transition probabilities

<img src="../image-39.png" width="500px" /><div/>

<img src="../image-40.png" width="500px" /><div/>


S=possible next states, P=transition probabilities

So action-value is the immediate reward for taking that action, plus the expected value of the next state, over all possible successor states

-----

Expand diagram to 2 steps to get MDP bellmann equation

<img src="../image-41.png" width="500px" /><div/>

<img src="../image-42.png" width="500px" /><div/>

---- 
also start from an action node:

<img src="../image-43.png" width="500px" /><div/>


And expand the action-value function by substituting in the state-value function:

<img src="../image-44.png" width="500px" /><div/>
-----
SO we get the **value function in terms of itself**, giving us the **MDP Bellman Equation**.


<img src="../image-45.png" width="500px" /><div/>

It implements this formula:

<img src="../image-46.png" width="500px" /><div/>




In [ ]:
import numpy as np

# ---------- MDP definition (dicts) ----------

S = list(state_names)  # ["C1", "C2", "C3", "FB", "Sleep"]
terminal_states_mdp = {"Sleep"}

A = {
    "C1": ["study", "facebook"],
    "C2": ["study", "sleep"],
    "C3": ["study", "pub"],
    "FB": ["quit", "facebook"],
    "Sleep": [],
}

# Transition model: P[(s,a)] = [(prob, s_next), ...]
P_sa = {
    ("C1", "study"): [(1.0, "C2")],
    ("C1", "facebook"): [(1.0, "FB")],
    ("C2", "study"): [(1.0, "C3")],
    ("C2", "sleep"): [(1.0, "Sleep")],
    # Study in C3 gives +10 then day ends
    ("C3", "study"): [(1.0, "Sleep")],
    # Pub in C3 transitions back to class states
    ("C3", "pub"): [(0.2, "C1"), (0.4, "C2"), (0.4, "C3")],
    ("FB", "quit"): [(1.0, "C1")],
    ("FB", "facebook"): [(1.0, "FB")],
}

# Reward model: R[(s,a)] = immediate reward for taking action a in s
R_sa = {
    ("C1", "study"): -2.0,
    ("C1", "facebook"): -1.0,
    ("C2", "study"): -2.0,
    ("C2", "sleep"): 0.0,
    ("C3", "study"): 10.0,
    ("C3", "pub"): 1.0,
    ("FB", "quit"): 0.0,
    ("FB", "facebook"): -1.0,
}


def actions(s):
    return A[s]


def transitions_sa(s, a):
    return P_sa[(s, a)]


def reward_sa(s, a):
    return float(R_sa[(s, a)])


# ---------- Bellman operators ----------


def bellman_expectation_backup_v(V, pi, gamma):
    """Bellman expectation equation for V under policy pi.

    V(s) = Σ_a π(a|s) Σ_{s'} P(s'|s,a) [ r(s,a) + γ V(s') ]
    """
    V_new = {}
    for s in S:
        if s in terminal_states_mdp:
            V_new[s] = 0.0
            continue
        v = 0.0
        for a, p_a in pi[s].items():
            q_sa = 0.0
            for p, s_next in transitions_sa(s, a):
                q_sa += p * (reward_sa(s, a) + gamma * V[s_next])
            v += p_a * q_sa
        V_new[s] = v
    return V_new


def q_from_v(V, gamma):
    """Compute Q(s,a) from V: Q(s,a) = Σ_{s'} P(s'|s,a) [ r(s,a) + γ V(s') ]."""
    Q = {s: {} for s in S}
    for s in S:
        if s in terminal_states_mdp:
            continue
        for a in actions(s):
            q = 0.0
            for p, s_next in transitions_sa(s, a):
                q += p * (reward_sa(s, a) + gamma * V[s_next])
            Q[s][a] = q
    return Q


# ---------- Dynamic programming algorithms ----------


def normalize_policy(pi):
    """Ensure pi[s] is a valid distribution over actions(s)."""
    out = {}
    for s in S:
        if s in terminal_states_mdp:
            out[s] = {}
            continue
        if s not in pi or not pi[s]:
            # default uniform
            acts = actions(s)
            out[s] = {a: 1.0 / len(acts) for a in acts}
            continue

        # keep only valid actions
        probs = {a: float(p) for a, p in pi[s].items() if a in set(actions(s))}
        total = sum(probs.values())
        if total <= 0:
            acts = actions(s)
            out[s] = {a: 1.0 / len(acts) for a in acts}
        else:
            out[s] = {a: p / total for a, p in probs.items()}
    return out


V^pi (uniform random):
     C1: -1.307692
     C2:  2.692308
     C3:  7.384615
     FB: -2.307692
  Sleep:  0.000000

V* (optimal):
     C1:  6.000000
     C2:  8.000000
     C3:  10.000000
     FB:  6.000000
  Sleep:  0.000000

Greedy optimal policy π*:
     C1 -> study
     C2 -> study
     C3 -> study
     FB -> quit

Q* (optimal action-values):
     C1: study=6.000000, facebook=5.000000
     C2: study=8.000000, sleep=0.000000
     C3: study=10.000000, pub=9.400000
     FB: quit=6.000000, facebook=5.000000


In [16]:
def policy_evaluation(pi, gamma=1.0, theta=1e-12, max_iters=100_000):
    """Iterative policy evaluation using the Bellman expectation backup."""
    pi = normalize_policy(pi)
    V = {s: 0.0 for s in S}

    for _ in range(max_iters):
        V_new = bellman_expectation_backup_v(V, pi, gamma)
        delta = max(abs(V_new[s] - V[s]) for s in S)
        V = V_new
        if delta < theta:
            break

    Q = q_from_v(V, gamma)
    return V, Q


# Uniform random policy (matches the earlier MDP->MRP conversion)
pi_uniform = {s: {a: 1.0 / len(actions(s)) for a in actions(s)} for s in S}

V_pi, Q_pi = policy_evaluation(pi_uniform, gamma=1.0, theta=1e-12)
print("V^pi (uniform random):")
for s in S:
    print(f"  {s:>5}: {V_pi[s]: .6f}")


V^pi (uniform random):
     C1: -1.307692
     C2:  2.692308
     C3:  7.384615
     FB: -2.307692
  Sleep:  0.000000


In [18]:
def value_iteration(gamma=1.0, theta=1e-12, max_iters=100_000):
    """Value iteration using the Bellman optimality backup.

    V(s) = max_a Σ_{s'} P(s'|s,a)[ r(s,a) + γ V(s') ]
    """
    V = {s: 0.0 for s in S}

    for _ in range(max_iters):
        delta = 0.0
        V_new = dict(V)
        for s in S:
            if s in terminal_states_mdp:
                V_new[s] = 0.0
                continue
            best = -float("inf")
            for a in actions(s):
                q = 0.0
                for p, s_next in transitions_sa(s, a):
                    q += p * (reward_sa(s, a) + gamma * V[s_next])
                best = max(best, q)
            delta = max(delta, abs(best - V[s]))
            V_new[s] = best
        V = V_new
        if delta < theta:
            break

    Q = q_from_v(V, gamma)

    # Greedy deterministic policy from Q*
    pi_star = {}
    for s in S:
        if s in terminal_states_mdp:
            pi_star[s] = {}
            continue
        best_a = max(Q[s].items(), key=lambda kv: kv[1])[0]
        pi_star[s] = {a: (1.0 if a == best_a else 0.0) for a in actions(s)}

    return V, Q, pi_star


# ---------- Run on the same example ----------

V_star, Q_star, pi_star = value_iteration(gamma=1.0, theta=1e-12)
print("\nV* (optimal):")
for s in S:
    print(f"  {s:>5}: {V_star[s]: .6f}")



V* (optimal):
     C1:  6.000000
     C2:  8.000000
     C3:  10.000000
     FB:  6.000000
  Sleep:  0.000000


In [17]:
print("\nGreedy optimal policy π*:")
for s in S:
    if s in terminal_states_mdp:
        continue
    best_a = max(pi_star[s].items(), key=lambda kv: kv[1])[0]
    print(f"  {s:>5} -> {best_a}")

print("\nQ* (optimal action-values):")
for s in S:
    if s in terminal_states_mdp:
        continue
    items = ", ".join(f"{a}={Q_star[s][a]:.6f}" for a in actions(s))
    print(f"  {s:>5}: {items}")



Greedy optimal policy π*:
     C1 -> study
     C2 -> study
     C3 -> study
     FB -> quit

Q* (optimal action-values):
     C1: study=6.000000, facebook=5.000000
     C2: study=8.000000, sleep=0.000000
     C3: study=10.000000, pub=9.400000
     FB: quit=6.000000, facebook=5.000000



### Optimal Value Functions

<img src="../image-47.png" width="500px" /><div/>

<img src="../image-48.png" width="500px" /><div/>


<img src="../image-49.png" width="500px" /><div/>




### Optimal policies

<img src="../image-50.png" width="500px" /><div/>

<img src="../image-51.png" width="500px" /><div/>

<img src="../image-52.png" width="500px" /><div/>


<img src="../image-53.png" width="500px" /><div/>


#### Finding The Optimal Action-Value Function

<img src="../image-54.png" width="500px" /><div/>



<img src="../image-56.png" width="500px" /><div/>

<img src="../image-56.png" width="500px" /><div/>

<img src="../image-57.png" width="500px" /><div/>

<img src="../image-58.png" width="500px" /><div/>

<img src="../image-59.png" width="500px" /><div/>
